# Shopify Summer 2022 Data Science Intern Challenge

## Question 1

On Shopify, we have exactly 100 sneaker shops, and each of these shops sells only one model of shoe. We want to do some analysis of the average order value (AOV). When we look at orders data over a 30 day window, we naively calculate an AOV of $3145.13. Given that we know these shops are selling sneakers, a relatively affordable item, something seems wrong with our analysis.

### Part a. 

Think about what could be going wrong with our calculation. Think about a better way to evaluate this data. 

### Part a. Solution

Prior to looking into the dataset, I've spent some time thinking about what could be going wrong with the calculation and I hypothesize that the issue may be caused by one of the following reasons:

- an incorrect calculation
    - an incorrect calculation of the right column (sum rather than average, etc.)
    - an average of the wrong column
- an uncleaned dataset
    - incorrect values in the dataset (misplaced decimals, etc.)
    - incorrect units
- outliers
    - correct outlying data that is raising the average
    - fraudulent outlying data that is raising the average


Now it's time to get into the data to try to validate some of these theories.

In [109]:
# imports
import pandas as pd

In [110]:
# get data
file = "2019 Winter Data Science Intern Challenge Data Set - Sheet1.csv"
df = pd.read_csv(file)

In [111]:
# view data
df.head()

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
0,1,53,746,224,2,cash,2017-03-13 12:36:56
1,2,92,925,90,1,cash,2017-03-03 17:38:52
2,3,44,861,144,1,cash,2017-03-14 4:23:56
3,4,18,935,156,1,credit_card,2017-03-26 12:43:37
4,5,18,883,156,1,credit_card,2017-03-01 4:35:11


Looking at the dataset, it appears that the Average Order Value (AOV) should simply be calculated by finding the mean value of the *order_amount* column.

In [112]:
# Average Order Value (AOV)
df['order_amount'].mean()

3145.128

It appears that the mean value of the *order_amount* column is indeed 3145.13 so we can rule out an incorrect calculation being the reason for this unexpectedly large value.

Now I will take a deeper look at the dataset to see if perhaps the dataset being uncleaned is the reason for the large AOV.

In [113]:
# dataset info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   order_id        5000 non-null   int64 
 1   shop_id         5000 non-null   int64 
 2   user_id         5000 non-null   int64 
 3   order_amount    5000 non-null   int64 
 4   total_items     5000 non-null   int64 
 5   payment_method  5000 non-null   object
 6   created_at      5000 non-null   object
dtypes: int64(5), object(2)
memory usage: 273.6+ KB


Looking at the dataset info, it can be seen that the *order_amount* column is filled with all int64 values and has no missing values. Let's check for duplicates.

In [114]:
# check for duplicates
df.duplicated().sum()

0

No duplicates in the dataset.

In [115]:
# dataset statistics
df.describe()

,order_id,shop_id,user_id,order_amount,total_items
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000
mean,2500.500000,50.078800,849.092400,3145.128000,8.78720
std,1443.520003,29.006118,87.798982,41282.539349,116.32032
min,1.000000,1.000000,607.000000,90.000000,1.00000
25%,1250.750000,24.000000,775.000000,163.000000,1.00000
50%,2500.500000,50.000000,849.000000,284.000000,2.00000
75%,3750.250000,75.000000,925.000000,390.000000,3.00000
max,5000.000000,100.000000,999.000000,704000.000000,2000.00000


Furthermore, looking at the dataset statistics, it can be seen that some interesting things are happening in the dataset. Specifically, the max values in the *order_amount* and *total_items* columns are higher than one might expect.

Before investigating these extreme values though, I must finish investigating the dataset to ensure there are no errors. 

From the problem definition it is known that each shop only sells one product which should have a single unique value. To ensure that this is held true throughout the dataset a new column *item_price* can be created.

In [116]:
# create item price column
df['item_price'] = df['order_amount']/df['total_items']

In [117]:
# check how many unique item prices each shop has
df.groupby('shop_id').nunique().sort_values(by = 'item_price', ascending = False)

,order_id,user_id,order_amount,total_items,payment_method,created_at,item_price
shop_id,,,,,,,
1,44,42,4,4,3,44,1
64,43,42,3,3,3,43,1
74,38,37,5,5,3,38,1
73,58,50,4,4,3,58,1
72,46,42,4,4,3,46,1
...,...,...,...,...,...,...,...
31,47,47,4,4,3,47,1
30,56,48,4,4,3,56,1
29,58,50,5,5,3,58,1


From the table above we can confidently say that each shop sells a single model of shoe at a single price. Now its time to investigate which shops are selling such expensive shoes!

In [118]:
# check shoe prices
df.groupby('shop_id').mean().sort_values(by = 'item_price', ascending = False)

,order_id,user_id,order_amount,total_items,item_price
shop_id,,,,,
78,2663.021739,867.739130,49213.043478,1.913043,25725.0
42,2441.921569,758.588235,235101.490196,667.901961,352.0
12,2555.415094,844.433962,352.698113,1.754717,201.0
89,2833.754098,829.803279,379.147541,1.934426,196.0
99,2386.000000,846.166667,339.444444,1.740741,195.0
...,...,...,...,...,...
53,2393.411765,858.544118,214.117647,1.911765,112.0
100,2356.525000,852.325000,213.675000,1.925000,111.0
32,2299.071429,856.809524,189.976190,1.880952,101.0


25725.0 seems a bit expensive for a pair of shoes if you ask me. Let us investigate shop 78 further to see who is willing to pay so much for a pair shoes.

In [119]:
# get shop 78 data
shop_78 = df[df['shop_id'] == 78]

In [120]:
# shop 78 data
shop_78

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at,item_price
160,161,78,990,25725,1,credit_card,2017-03-12 5:56:57,25725.0
490,491,78,936,51450,2,debit,2017-03-26 17:08:19,25725.0
493,494,78,983,51450,2,cash,2017-03-16 21:39:35,25725.0
511,512,78,967,51450,2,cash,2017-03-09 7:23:14,25725.0
617,618,78,760,51450,2,cash,2017-03-18 11:18:42,25725.0
691,692,78,878,154350,6,debit,2017-03-27 22:51:43,25725.0
1056,1057,78,800,25725,1,debit,2017-03-15 10:16:45,25725.0
1193,1194,78,944,25725,1,debit,2017-03-16 16:38:26,25725.0
1204,1205,78,970,25725,1,credit_card,2017-03-17 22:32:21,25725.0
1259,1260,78,775,77175,3,credit_card,2017-03-27 9:27:20,25725.0


In [121]:
# check how many people are buying shoes from shop 78
shop_78['user_id'].nunique()

45

It appears that many people, 45 to be specific, are willing to pay 25725.0 for a pair of shoes... or so it appears in the dataset.

Although it cannot be deduced why the shoe price at shop 78 is so high with the information available, my guess would be that perhaps this value is a mistake and has a misplaced decimal or perhaps there is some fraudulence at play here. Either way, perhaps it is best that we remove the data from shop 78 for now.

In [122]:
# remove shop 78's data
df = df[df['shop_id'] != 78]

Now let's take a look at the datasets statistics again.

In [123]:
# dataset statistics
df.describe()

,order_id,shop_id,user_id,order_amount,total_items,item_price
count,4954.000000,4954.000000,4954.000000,4954.000000,4954.000000,4954.000000
mean,2498.990916,49.819540,848.919257,2717.367784,8.851029,152.475575
std,1444.498907,29.014845,87.846007,41155.996469,116.857286,31.260218
min,1.000000,1.000000,607.000000,90.000000,1.000000,90.000000
25%,1248.250000,24.000000,775.000000,163.000000,1.000000,132.000000
50%,2494.500000,50.000000,849.000000,284.000000,2.000000,153.000000
75%,3750.750000,74.000000,925.000000,390.000000,3.000000,168.000000
max,5000.000000,100.000000,999.000000,704000.000000,2000.000000,352.000000


The *item_price* column statistics seem reasonable now; however, it can be seen that there are still some abnormally large values in the *order_amount* and *total_items* columns. Let us investigate this further.

In [124]:
# check order_amount statistics
df.groupby('shop_id').mean().sort_values(by = 'order_amount', ascending = False)

,order_id,user_id,order_amount,total_items,item_price
shop_id,,,,,
42,2441.921569,758.588235,235101.490196,667.901961,352.0
50,2162.431818,834.500000,403.545455,2.090909,193.0
90,2694.163265,839.857143,403.224490,2.265306,178.0
38,2639.714286,837.800000,390.857143,2.057143,190.0
81,2685.627119,835.881356,384.000000,2.169492,177.0
...,...,...,...,...,...
53,2393.411765,858.544118,214.117647,1.911765,112.0
100,2356.525000,852.325000,213.675000,1.925000,111.0
32,2299.071429,856.809524,189.976190,1.880952,101.0


In [125]:
# check total_items statistics
df.groupby('shop_id').mean().sort_values(by = 'total_items', ascending = False)

,order_id,user_id,order_amount,total_items,item_price
shop_id,,,,,
42,2441.921569,758.588235,235101.490196,667.901961,352.0
37,2657.958333,847.416667,340.208333,2.395833,142.0
24,2524.290909,840.236364,320.727273,2.290909,140.0
90,2694.163265,839.857143,403.224490,2.265306,178.0
10,2426.358491,868.075472,332.301887,2.245283,148.0
...,...,...,...,...,...
25,2651.375000,846.708333,232.916667,1.791667,130.0
47,2520.659574,863.617021,259.148936,1.787234,145.0
12,2555.415094,844.433962,352.698113,1.754717,201.0


It appears that shop 42 seems to be the source of these abnormally large values, so perhaps we should take a deeper look at that shop and it's sales to see who is making these large purchases.

In [126]:
# get shop 42 data
shop_42 = df[df['shop_id'] == 42]

In [127]:
# check largest orders at shop 42
shop_42.sort_values(by = 'order_amount', ascending = False)

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at,item_price
15,16,42,607,704000,2000,credit_card,2017-03-07 4:00:00,352.0
1436,1437,42,607,704000,2000,credit_card,2017-03-11 4:00:00,352.0
4868,4869,42,607,704000,2000,credit_card,2017-03-22 4:00:00,352.0
4646,4647,42,607,704000,2000,credit_card,2017-03-02 4:00:00,352.0
4056,4057,42,607,704000,2000,credit_card,2017-03-28 4:00:00,352.0
3332,3333,42,607,704000,2000,credit_card,2017-03-24 4:00:00,352.0
2969,2970,42,607,704000,2000,credit_card,2017-03-28 4:00:00,352.0
2835,2836,42,607,704000,2000,credit_card,2017-03-28 4:00:00,352.0
2297,2298,42,607,704000,2000,credit_card,2017-03-07 4:00:00,352.0
1602,1603,42,607,704000,2000,credit_card,2017-03-17 4:00:00,352.0


It appears that user 607 seems to be the source of all the abnormally large orders from shop 42. Let us investigate this user further.

In [128]:
# get user 607 data
user_607 = df[df['user_id'] == 607]

In [129]:
# user 607 data
user_607

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at,item_price
15,16,42,607,704000,2000,credit_card,2017-03-07 4:00:00,352.0
60,61,42,607,704000,2000,credit_card,2017-03-04 4:00:00,352.0
520,521,42,607,704000,2000,credit_card,2017-03-02 4:00:00,352.0
1104,1105,42,607,704000,2000,credit_card,2017-03-24 4:00:00,352.0
1362,1363,42,607,704000,2000,credit_card,2017-03-15 4:00:00,352.0
1436,1437,42,607,704000,2000,credit_card,2017-03-11 4:00:00,352.0
1562,1563,42,607,704000,2000,credit_card,2017-03-19 4:00:00,352.0
1602,1603,42,607,704000,2000,credit_card,2017-03-17 4:00:00,352.0
2153,2154,42,607,704000,2000,credit_card,2017-03-12 4:00:00,352.0
2297,2298,42,607,704000,2000,credit_card,2017-03-07 4:00:00,352.0


Investigating user 607 it appears they made 17 orders, all from shop 42. Each order involved the purchase of 
2000 items for a total amount of 704000. Furthermore, each purchase was made by credit card.

Credit card purchases of this size to an online shoe store within a timeframe of a month seems like possible fraudulent behaviour. Perhaps it would be best to remove user 607's data from the dataset.

In [130]:
# remove user 607's data from the dataset
df = df[df['user_id'] != 607]

Now let's take a look at the datasets statistics again.

In [131]:
# dataset statistics
df.describe()

,order_id,shop_id,user_id,order_amount,total_items,item_price
count,4937.000000,4937.000000,4937.000000,4937.000000,4937.000000,4937.000000
mean,2499.551347,49.846465,849.752279,302.580514,1.994734,151.788536
std,1444.069407,29.061131,86.840313,160.804912,0.982821,29.034215
min,1.000000,1.000000,700.000000,90.000000,1.000000,90.000000
25%,1248.000000,24.000000,775.000000,163.000000,1.000000,132.000000
50%,2497.000000,50.000000,850.000000,284.000000,2.000000,153.000000
75%,3751.000000,74.000000,925.000000,387.000000,3.000000,166.000000
max,5000.000000,100.000000,999.000000,1760.000000,8.000000,352.000000


These values seem much more reasonable. As can be seen, the mean value of the *order_amount* column in this cleaned dataset is now 302.58.

### Part b.

What metric would you report for this dataset?

### Part b. Solution

Judging from the analysis performed above, it seems that shop 78 and user 607 were outliers in the dataset and were the reason behind the abnormally large AOV that was initially calculated. With that being said, the mean value of the *order_value* column in the cleaned dataset can be used to describe the AOV; however, this value is caculated under the assumption that shop 78 and user 607 do not belong in the dataset, which we cannot say for certain.

Alternatively, if we do not want to make the assumption that shop 78 and user 607 do not belong in the dataset, we can use the median value of the *order_amount* column as the median is less susceptible to the influence of outliers compared to the mean. Furthermore, it can be seen from the descriptive statistics tables generated in the notebook that the median value of the *order_amount* column is the same in the original dataset as in the cleaned dataset, hence, we can be confident that it accurately portrays the spending habits of a typical consumer in this dataset.

### Part c.

What is its value?

### Part c. Solution

In [132]:
# calculate median order value
df['order_amount'].median()

284.0

The median order_amount in the dataset is 284.

## Question 2

 For this question you’ll need to use SQL. Follow this link to access the data set required for the challenge. Please use queries to answer the following questions. Paste your queries along with your final numerical answers below.

### Part a.

How many orders were shipped by Speedy Express in total?

### Part a. Solution

Running the following query we can determine that Speedy Express has shipped **54** orders in total.

In [133]:
SELECT COUNT(*)
FROM Orders INNER JOIN Shippers ON Orders.ShipperID=Shippers.ShipperID
WHERE ShipperName == "Speedy Express"


SyntaxError: invalid syntax (<ipython-input-133-80ade1dfa4c5>, line 1)

### Part b.

What is the last name of the employee with the most orders?

### Part b. Solution

Running the following query we can determine that the last name of the employee with the most orders was **Peacock**.

In [ ]:
SELECT LastName, COUNT(*)
FROM Employees INNER JOIN Orders ON Employees.EmployeeID=Orders.EmployeeID
GROUP BY LastName
ORDER BY COUNT(*) DESC
LIMIT 1

SyntaxError: invalid syntax (<ipython-input-105-4370f10986c7>, line 1)

### Part c.

What product was ordered the most by customers in Germany?

### Part c. Solution

Running the following query we can determine that the product most ordered by customers in Germany was **Gorgonzola Telino**.

In [ ]:
SELECT ProductName, COUNT(*)
FROM Products
INNER JOIN OrderDetails ON Products.ProductID=OrderDetails.ProductID
INNER JOIN Orders ON OrderDetails.OrderID=Orders.OrderID
INNER JOIN Customers ON Orders.CustomerID=Customers.CustomerID
WHERE Country = "Germany"
GROUP BY ProductName
ORDER BY COUNT(*) DESC
LIMIT 1

SyntaxError: invalid syntax (<ipython-input-108-245046921d93>, line 1)